## ResNet + CIFAR10

Как получить 90%+ точности для Resnet
...


Augmentation  + manual decrease LR 

#Работа с реальными данными

Проблемы:

- нехватка данных
- недостаток размеченных данных
- не качественная разметка
- не сбалансированность датасета


### ImageFolder

Создадим датасет из своих данных для этого достаточно разложить изображения по папкам и использовать класс ImgeFolder

In [ ]:
!wget  http://fmb.images.gan4x4.ru/hse/bt_dataset3.zip
!unzip -q bt_dataset3.zip

In [ ]:
!ls bike/bike_type/train
!ls bike/bike_type/val

In [ ]:
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import numpy as np
import matplotlib.pyplot as plt

train_dataset = ImageFolder("/content/bike/bike_type/train")
val_dataset = ImageFolder("/content/bike/bike_type/val")

fig=plt.figure(figsize=(24, 6))
for i in range(1, 2*7 +1):
    img = val_dataset[i][0]
    fig.add_subplot(2, 7, i)
    plt.imshow(np.asarray(img))
plt.show()


Обратите внимание: на многих кадрах один и тот же велосипед. Хорошо ли это? Что будет если часть кадров попадет в обучающую, а часть в теренировочную выборку.

##Дисбаланс классов

In [ ]:
print("Classes: ",val_dataset.classes)
print("Sizes train:",len(train_dataset), 'val', len(val_dataset))

У imageFolder есть свойство classes которое запослняется в соответствии с названиями папок.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def class_hist(dataset):
  plt.figure() 
  unique, counts = np.unique(dataset.targets, return_counts=True)
  ax = plt.bar(unique, counts)
  plt.title('Train objects')
  plt.xticks(unique, dataset.classes)
  plt.show()
  return counts

class_hist(train_dataset)
 

Что делать?

1. Использовать адекватные метрики:
- F1_score
- PR_Curve
- Confusion matrix



```
# targets and preds must be calculated during training

from sklearn.metrics import confusion_matrix
import pandas as pd

conf_matrix = pd.DataFrame(confusion_matrix(targets, preds))

conf_matrix.columns = dataset_val.classes
conf_matrix.index = dataset_val.classes

conf_matrix = conf_matrix.rename_axis('Real')
conf_matrix = conf_matrix.rename_axis('Predicted', axis='columns')

conf_matrix
```

<img src ="http://edunet.kea.su/repo/src/L11_Transfer_learning/img/gan/conf_matrix.png" width="600">



### Использовать веса в Loss - функции

https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html?highlight=crossentropy#torch.nn.CrossEntropyLoss

Как следует из описания к CrossEntropyLoss можно добавить веса классов. Сравним loss для с весам и без для оних и тех же данных.

In [ ]:
import torch
# Without weights
scores = torch.tensor([[2.,30.],[2.,30.]]) # Scores for two samples batch
target = torch.tensor([0,1]) # First sample belongs to class 0 second to 1 and firse was misclassified
weights = torch.tensor([1,1],dtype = torch.float32)
criterion = torch.nn.CrossEntropyLoss( weight = weights,reduction = 'mean')  #'mean'
criterion(scores,target)


Добавим к первому классу вес 10. Это условно соответствует ситуации с BMX велосипедами в нашем датасете: их примерно в 10 раз больше чем MTB

In [ ]:
weights = torch.tensor([10,1],dtype = torch.float32)
criterion = torch.nn.CrossEntropyLoss( weight = weights,reduction = 'mean')  #'mean'
criterion(scores,target)


Лосс вырос так как класс на котором возникла ошибка оказался редким.

Рассчет весов

In [ ]:
#numpy
import numpy as np
_, counts = np.unique(train_dataset.targets, return_counts=True)
weights= np.max(counts) / counts
weights = torch.FloatTensor(weights)
print('Веса классов: ', weights)

In [ ]:
#torch
# https://pytorch.org/docs/stable/generated/torch.unique.html
_, counts = torch.unique(torch.tensor(train_dataset.targets),return_counts = True)
print(counts.max())
weights = counts.max() / counts
print('Классы: ',train_dataset.classes)
print('Веса классов: ', weights)

In [ ]:
# Sklearn
#https://scikit-learn.org/stable/modules/generated/sklearn.utils.class_weight.compute_class_weight.html
from sklearn.utils import class_weight
weights = class_weight.compute_class_weight('balanced',np.unique(train_dataset.targets),train_dataset.targets)
print(weights)
weights / min(weights) # The same as before


#### Выборка с повышением или с понижением (Up-sample or Down-sample): одно из решений проблемы - сбалансировать данные.

  

```
  Это может быть сделано либо за счет увеличения частоты класса меньшинства, либо за счет уменьшения частоты класса большинства с помощью методов случайной или кластерной выборки.

    Выбор между избыточной или недостаточной выборкой и случайным или кластеризованным определяется бизнес-контекстом и размером данных.

    Обычно `upsampling` предпочтителен, когда общий размер данных небольшой, а понижающая дискретизация полезна, когда у нас есть большой объем данных. Точно так же случайная или кластерная выборка определяется тем, насколько хорошо распределены данные.
```
<img src ="http://edunet.kea.su/repo/src/L11_Transfer_learning/img/L11_34.png" width="600">

Фактически мы либо удаляем часть объектов одного из классов либо копируем.

## Недостаток данных

In [ ]:
class_hist(val_dataset)


На 3-х изображениях мы не сможем оценить точность. Давайте на них посмотрим

Очевидно что выкидывать тут уже нечего, и 100 копий одного велосипеда не решат проблемму. В данном случае можно переместить несколько велосипедов из тренировочной выборки в валидационную но поблему дефицита BMX-ов в целом это не решит. 

#### Генерация синтетических данных:

 хотя `upsampling` или `downsampling` помогает сбалансировать данные, дублирование данных увеличивает вероятность переобучения.

Другой подход к решению этой проблемы - создание синтетических данных с помощью данных о классе меньшинств.

Для табличных данных можно использовать методы 

[Synthetic Minority Over-sampling Technique (SMOTE)](https://rikunert.com/SMOTE_explained) или Modified- SMOTE - два таких метода, которые генерируют синтетические данные.

Проще говоря, SMOTE берет точки данных класса меньшинства и создает новые точки данных, которые лежат между любыми двумя ближайшими точками данных, соединенными прямой линией.

Для этого алгоритм вычисляет расстояние между двумя точками данных в пространстве признаков, умножает расстояние на случайное число от 0 до 1 и помещает новую точку данных на этом новом расстоянии от одной из точек данных, используемых для определения расстояния.

<img src ="http://edunet.kea.su/repo/src/L11_Transfer_learning/img/L11_35.png" width="700">

## Аугментации

Применяются для изображений с той же целью: получить дополнительные данные. 

Сам термин пришел из музыки:

Аугмента́ция (позднелат. augmentatio — увеличение, расширение) — техника ритмической композиции в старинной музыке.

<img src ="http://edunet.kea.su/repo/src/L11_Transfer_learning/img/L11_12.png" width="700">

Собственно механизм трансформаций torcvision который мы использовали с первого занятия, в основном предназначен для аугментации данных.

<img src ="http://edunet.kea.su/repo/src/L11_Transfer_learning/img/L11_13.png" width="700">

#### Создание собственных аугментаций

В том числе могут применяться в том числе  и к меткам!

#### Целесообразность

#### Применение большого количества аугментаций может испортить изображение

In [ ]:
RandomChoice и/или RandomOrder из torchvision.transforms

#### Аугментация как регуляризация
- помогает бороться с переобучением


#### Albumentation

https://github.com/albumentations-team/albumentations


### Unsupervised Learning

## Transfer learning

Для таких типовых задач, как классификация изображений, можно воспользоваться готовой архитектурой (AlexNet, VGG, Inception, ResNet и т.д.) и обучить нейросеть на своих данных. Реализации таких сетей с помощью различных фреймворков уже существуют, так что на данном этапе можно использовать одну из них как черный ящик, не вникая глубоко в принцип её работы.

Однако, глубокие нейронные сети требовательны к большим объемам данных для сходимости обучения. И зачастую, в нашей частной задаче недостаточно данных для того, чтобы хорошо натренировать все слои нейросети. `Transfer Learning` решает эту проблему. Зачем обучать сеть заново, если можно использовать уже обученную на миллионе изображений и дообучить на свой датасет?

В PyTorch есть много предобученных сетей: [TORCHVISION.MODELS](https://pytorch.org/vision/stable/models.html)

- AlexNet
- VGG
- ResNet
- SqueezeNet
- DenseNet
- Inception v3
- GoogLeNet
- ShuffleNet v2
- MobileNetV2
- MobileNetV3
- ResNeXt
- Wide ResNet
- MNASNet

Для этого,  нужно отключить какие-то промежуточные слои. Тогда можно использовать то, что называется `Fine turning` - не нужно обучать всю модель, а достаточно только ее новую часть.

Зачастую в конце классификационных сетей используется полносвязный слой. Так как мы заменили этот слой, использовать предобученные веса для него уже не получится. Придется тренировать его с нуля, инициализировав его веса случайными значениями. Веса для всех остальных слоев мы загружаем из предобученной модели.

<img src ="http://edunet.kea.su/repo/src/L11_Transfer_learning/img/L11_21.png" width="700">

Мы уже делали это когда сравнивали Resnet собственного изготовления библиотечной реализацией. Нам требовалось изменить колчество классов и мы просто заменяли у можели последний слой.

In [ ]:
from torchvision.models import resnet18

model = resnet18()
print(model)

Смотрим на вывод, линейный слой, и заменяем его другим с тем же количеством входов и нужным нам количеством выходов


In [ ]:
from torch import nn
model.fc = nn.Linear(model.fc.in_features, 10)

imagenet_input = torch.randn([1,3,224,224])

out = model(imagenet_input)
print(out)

## Заморозка весов




In [ ]:
for tag, param in model.named_parameters():
  if not 'layer4.' in tag and not 'fc' in tag:
  #if not any(map(tag.__contains__, ['layer4.','fc'])): another way
    param.requires_grad = False
  print(tag,type(param),param.shape,param.requires_grad)

In [ ]:
from torch import optim

optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# or 

params_to_update = list(model.fc.parameters()) + list(model.layer4.parameters())
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

#### Замена и удаление произвольных слоев

При отправке тензора с высотой и шириной 32x32 возникает ошибка

In [ ]:
cifar10_input = torch.randn([1,3,32,32])
out = model(cifar10_input)
print(out)

Заменим 'stem' слои в начале сети, отвечающие за аггресивное сжатие изображения.

P.S. Это потребуется при выполнении практической работы.

In [ ]:
from torch import nn
model.conv1 = nn.Conv2d(3,64,kernel_size=(5, 5),stride = 1, padding =2, bias=False)
model.maxpool = nn.Identity()
out = model(cifar10_input)
print(out)


## Рекомендации по обучению

In [ ]:
Форматы изображений (OpenCv, Pillow ...)

### Сохранение весов

#Few shot learning

## Распознавание лиц

## Сиамские сети

### Косинусное расстояние

### Contrastive Loss

## Метрики

## Triplet Loss

## Предоброботка

### Ключевые точки

### Выравнивание

### MTCNN

### Кластеризация / Поиск

### Демонстрация